In [1]:
#working on parsing the data using pandas.read_csv()
import obspy

import pandas as pd

def parseRtergHtml(html):
    colNames1 = ["Eventname", "oDate", "oTime", "Lat.", "Long.", "Depth", "Me", "Txo",
                 "Ehf", "Ebb", "Mehf", "Ehf/Tr^3", "Nstats", "colon", "SRC", "iMag"]
    df1 = pd.read_csv(html, names = colNames1, skiprows = 1, nrows = 1, delim_whitespace=True)
    del df1["colon"]  #remove colon column
    df1["SRC"] = df1.iloc[0]["SRC"][5:7]
    iMagTypeVal = df1.iloc[0]["iMag"].split("=")[0]
    df1.insert(14, "iMagType", iMagTypeVal)
    df1["iMag"] = df1.iloc[0]["iMag"].split("=")[1][:-2]
    oTimeVal = df1["oDate"] + " " + df1["oTime"]
    del df1["oDate"]
    df1["oTime"] = pd.to_datetime(oTimeVal, utc=True)
    
    colNames2 = ["TACER_HF", "TACER_BB"]
    df2 = pd.read_csv(html, names = colNames2, skiprows = 2, nrows = 1, delim_whitespace=True, usecols=[7,10])
    
    colNames3 = ["junk", "Comment"]
    df3 = pd.read_csv(html, names = colNames3, skiprows = 3, nrows = 1, delimiter=":")
    del df3["junk"]
    
    colNames4 = ["mTime"]
    df4 = pd.read_csv(html, names = colNames4, skiprows = 7, nrows = 1, delimiter="?")
    df4["mTime"] = pd.to_datetime(df4["mTime"])
    
    colNames5 = ["junk", "iteration"]
    df5 = pd.read_csv(html, names = colNames5, skiprows = 8, nrows = 1, delimiter="=")
    del df5["junk"]
    df5["iteration"][0] = df5.iloc[0]["iteration"].split("<")[0]

    df = pd.concat([df1,df2,df3,df4, df5], axis=1)
    return df

df = parseRtergHtml("rterg_html_outs/21012303.html")
df.head()

,Eventname,oTime,Lat.,Long.,Depth,Me,Txo,Ehf,Ebb,Mehf,Ehf/Tr^3,Nstats,SRC,iMagType,iMag,TACER_HF,TACER_BB,Comment,mTime,iteration
0,21012303,2021-01-23 23:36:55+00:00,-62.0,-55.3,10.0,7.01,53,9.700000e+13,7.300000e+14,6.89,650000000.0,32,AT,M,7.3,27.0,26.0,this is our comment text,2021-01-28 14:25:57+00:00,A


In [2]:
# takes in a dataframe and parses it into xml format

import xml.etree.ElementTree as et

def toXML(df):
    
    name = str(df[df.columns[0]][0]) + ".xml"
    
    root = et.Element("root")
    tree = et.ElementTree(root)
    event = et.Element("event")
    root.append(event)
    
    for x in range(20):
        data = str(df[df.columns[x]][0])
        type = et.SubElement(event, df.columns[x]).text = data
    
    file = et.ElementTree(root)
    file.write(name, xml_declaration=True)
    return

toXML(df)

# don't run below here

In [1]:
#builds and empty dataframe

# old testing, do not run.

import pandas as pd

def buildDF():
    colNames = ["#Eventname", "Date", "Otime", "Lat.", "Long.", "Depth", "Me", "Tr", "Ehf", "Ebb",
                "Mehf", "Ehf/Tr^3", "Nstats : Comments", "TACER_HF", "TACER_BB", "COMMENTS"]
    df = pd.DataFrame(columns = colNames)
    return df

#uses a helper function to input data into the dataframe

def parseRtergHtml(df, html):
    data = readHtml(html)
    for x in range(16):
        df[df.columns[x]] = [data[x]]
    return

#creates a list of data from the html file that will be
#inputted into the dataframe

def readHtml(html):
    file = open(html, "r")
    file.readline()
    line2 = file.readline()
    dataList = line2.split()
    str = ""
    for x in range(3):
        str = str + " " + dataList.pop(13)
    dataList[12] = dataList[12] + str
    line3 = file.readline()
    tempList = line3.split()
    dataList.append(tempList[7])
    dataList.append(tempList[10])
    line4 = file.readline()
    dataList.append(line4[15:].strip())
    return dataList